In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, recall_score, mean_squared_error, mean_absolute_error, f1_score, confusion_matrix, precision_score, roc_curve, auc
from scipy.signal import savgol_filter
from tqdm import tqdm
from PyEMD import EEMD
import time

plt.rcParams['font.family'] = 'SimHei'
plt.rcParams['axes.unicode_minus'] = False
sns.set(style='whitegrid')

torch.manual_seed(42)
np.random.seed(42)


class Config:
    symbols = ['AAPL', 'MSFT', 'AMZN', 'GOOG', 'META', 'NVDA', 'TSLA']
    start_date = '2012-01-01'
    end_date = pd.Timestamp.now().strftime('%Y-%m-%d')
    seq_length = 20
    train_ratio = 0.8
    batch_size = 128
    epochs = 200
    lr = 5e-4
    weight_decay = 1e-4
    dropout = 0.2
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model_config = {
        'LSTM': {'hidden_size': 128, 'num_layers': 2},
        'GRU': {'hidden_size': 128, 'num_layers': 2}
    }


class DataProcessor:
    def __init__(self):
        self.scaler = StandardScaler()

    def get_data(self, symbol):
        max_retries = 3
        for attempt in range(max_retries):
            try:
                df = yf.download(symbol, start=Config.start_date, end=Config.end_date)
                return df[['Open', 'High', 'Low', 'Close', 'Volume']].dropna()
            except yf.YFRateLimitError:
                print(f"Rate limit reached for {symbol}. Retrying in 60 seconds...")
                time.sleep(60)
        print(f"Failed to download data for {symbol} after {max_retries} attempts.")
        return None

    def compute_rsi(self, prices, period=14):
        delta = prices.diff()
        gain = delta.where(delta > 0, 0).rolling(window=period).mean()
        loss = -delta.where(delta < 0, 0).rolling(window=period).mean()
        rs = gain / loss
        return 100 - (100 / (1 + rs))

    def compute_macd(self, prices, fast=12, slow=26, signal=9):
        ema_fast = prices.ewm(span=fast, adjust=False).mean()
        ema_slow = prices.ewm(span=slow, adjust=False).mean()
        macd = ema_fast - ema_slow
        signal_line = macd.ewm(span=signal, adjust=False).mean()
        return macd, signal_line

    def add_features(self, df):
        df['LogReturn'] = np.log(df['Close'] / df['Close'].shift(1))
        df['MA5'] = df['Close'].rolling(5).mean()
        df['MA20'] = df['Close'].rolling(20).mean()
        df['RSI'] = self.compute_rsi(df['Close'])
        macd, signal = self.compute_macd(df['Close'])
        df['MACD'] = macd
        df['MACD_Signal'] = signal
        df['DayOfWeek'] = df.index.dayofweek
        df['Month'] = df.index.month - 1
        return df.dropna()

    def decompose_with_eemd(self, series):
        eemd = EEMD()
        # 生成时间向量
        T = np.arange(len(series))
        imfs = eemd.emd(series, T)
        return imfs

    def plot_eemd_decomposition(self, symbol, series, imfs):
        plt.figure(figsize=(12, 8))
        plt.subplot(len(imfs) + 1, 1, 1)
        plt.plot(series, label='seires')
        plt.title(f'{symbol} EEMD')
        plt.legend()

        for i, imf in enumerate(imfs):
            plt.subplot(len(imfs) + 1, 1, i + 2)
            plt.plot(imf, label=f'IMF_{i}')
            plt.legend()

        plt.tight_layout()
        plt.savefig(f'{symbol}_eemd_decomposition.png')
        plt.close()

    def process_data(self, symbol, remove_pe=False, remove_imf=False, remove_tech=False):
        df = self.get_data(symbol)
        if df is None:
            return None, None
        df = self.add_features(df)
        # Smooth returns
        window_length = min(11, len(df['LogReturn']))
        df['LogReturn'] = savgol_filter(df['LogReturn'], window_length, 2)

        # 使用EEMD分解LogReturn
        log_return_series = df['LogReturn'].values
        imfs = self.decompose_with_eemd(log_return_series)
        self.plot_eemd_decomposition(symbol, log_return_series, imfs)

        for i, imf in enumerate(imfs):
            df[f'IMF_{i}'] = imf

        features = ['LogReturn', 'MA5', 'MA20', 'RSI', 'MACD', 'MACD_Signal', 'DayOfWeek', 'Month']
        if not remove_imf:
            for i in range(len(imfs)):
                features.append(f'IMF_{i}')
        if remove_tech:
            features = [f for f in features if f not in ['MA5', 'MA20', 'RSI', 'MACD', 'MACD_Signal']]
        target = 'LogReturn'
        df[features] = self.scaler.fit_transform(df[features])

        X, y = [], []
        for i in range(len(df) - Config.seq_length):
            X.append(df[features].values[i:i + Config.seq_length])
            y.append(df[target].values[i + Config.seq_length])
        return np.array(X), np.array(y)


class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, 1)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.dropout(out[:, -1, :])
        out = self.fc(out)
        return out


class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout):
        super().__init__()
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, 1)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out, _ = self.gru(x)
        out = self.dropout(out[:, -1, :])
        out = self.fc(out)
        return out


class Trainer:
    def __init__(self, model, model_name):
        self.model = model.to(Config.device)
        self.model_name = model_name
        self.criterion = nn.HuberLoss(delta=1.0)
        self.optimizer = optim.AdamW(model.parameters(), lr=Config.lr, weight_decay=Config.weight_decay)
        self.warmup_scheduler = optim.lr_scheduler.LinearLR(self.optimizer, start_factor=0.1, total_iters=10)
        self.cosine_scheduler = optim.lr_scheduler.CosineAnnealingLR(self.optimizer, T_max=Config.epochs - 10)
        self.best_loss = float('inf')
        self.early_stop_counter = 0

    def train_epoch(self, train_loader, epoch):
        self.model.train()
        total_loss = 0
        for X, y in train_loader:
            X, y = X.to(Config.device), y.to(Config.device)
            self.optimizer.zero_grad()
            outputs = self.model(X)
            loss = self.criterion(outputs.squeeze(), y)
            loss.backward()
            nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
            self.optimizer.step()
            total_loss += loss.item()
        if epoch < 10:
            self.warmup_scheduler.step()
        else:
            self.cosine_scheduler.step()
        return total_loss / len(train_loader)

    def evaluate(self, test_loader):
        self.model.eval()
        total_loss = 0
        preds, truths = [], []
        with torch.no_grad():
            for X, y in test_loader:
                X, y = X.to(Config.device), y.to(Config.device)
                outputs = self.model(X)
                loss = self.criterion(outputs.squeeze(), y)
                total_loss += loss.item()
                preds.extend(outputs.cpu().numpy().flatten())
                truths.extend(y.cpu().numpy().flatten())
        return total_loss / len(test_loader), np.array(preds), np.array(truths)

    def train(self, train_loader, test_loader):
        for epoch in range(Config.epochs):
            train_loss = self.train_epoch(train_loader, epoch)
            test_loss, preds, truths = self.evaluate(test_loader)
            if test_loss < self.best_loss:
                self.best_loss = test_loss
                torch.save(self.model.state_dict(), f'best_{self.model_name}.pth')
                self.early_stop_counter = 0
            else:
                self.early_stop_counter += 1
                if self.early_stop_counter >= 15:
                    print(f"Early stopping at epoch {epoch}")
                    break
            if epoch % 10 == 0:
                print(f"Epoch {epoch + 1}/{Config.epochs} | Train Loss: {train_loss:.4f} | Test Loss: {test_loss:.4f}")
        self.model.load_state_dict(torch.load(f'best_{self.model_name}.pth'))
        return preds, truths


class Evaluator:
    @staticmethod
    def calculate_metrics(preds, truths):
        pred_dir = (preds > 0).astype(int)
        true_dir = (truths > 0).astype(int)
        fpr, tpr, thresholds = roc_curve(true_dir, preds)
        roc_auc = auc(fpr, tpr)
        metrics = {
            'RMSE': np.sqrt(mean_squared_error(truths, preds)),
            'MAE': mean_absolute_error(truths, preds),
            'Accuracy': accuracy_score(true_dir, pred_dir),
            'Recall': recall_score(true_dir, pred_dir),
            'Sharpe': np.mean(preds) / np.std(preds) * np.sqrt(252) if np.std(preds) != 0 else 0,
            'Correlation': np.corrcoef(truths, preds)[0, 1],
            'F1': f1_score(true_dir, pred_dir),
            'ConfusionMatrix': confusion_matrix(true_dir, pred_dir),
            'Precision': precision_score(true_dir, pred_dir),
            'AUC': roc_auc
        }
        return metrics


def main():
    processor = DataProcessor()
    results = {}
    ablation_settings = [
        {'name': 'Full', 'remove_pe': False, 'remove_imf': False, 'remove_tech': False},
        {'name': 'No_PE', 'remove_pe': True, 'remove_imf': False, 'remove_tech': False},
        {'name': 'No_IMF', 'remove_pe': False, 'remove_imf': True, 'remove_tech': False},
        {'name': 'No_Tech', 'remove_pe': False, 'remove_imf': False, 'remove_tech': True}
    ]

    for symbol in Config.symbols:
        print(f"\n=== Processing {symbol} ===")
        symbol_results = {}
        for setting in ablation_settings:
            # 只传递 process_data 方法需要的参数
            relevant_params = {k: v for k, v in setting.items() if k in ['remove_pe', 'remove_imf', 'remove_tech']}
            X, y = processor.process_data(symbol, **relevant_params)
            if X is None or y is None:
                continue
            train_size = int(len(X) * Config.train_ratio)
            X_train, X_test = X[:train_size], X[train_size:]
            y_train, y_test = y[:train_size], y[train_size:]
            # 创建TensorDataset对象
            train_dataset = torch.utils.data.TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
            test_dataset = torch.utils.data.TensorDataset(torch.FloatTensor(X_test), torch.FloatTensor(y_test))
            # 使用TensorDataset对象创建DataLoader
            train_loader = DataLoader(train_dataset, batch_size=Config.batch_size, shuffle=True)
            test_loader = DataLoader(test_dataset, batch_size=Config.batch_size)
            model_results = {}
            input_size = X_train.shape[2]

            lstm = LSTMModel(input_size, **Config.model_config['LSTM'], dropout=Config.dropout)
            lstm_trainer = Trainer(lstm, f'LSTM_{setting["name"]}')
            lstm_preds, lstm_truths = lstm_trainer.train(train_loader, test_loader)
            model_results['LSTM'] = Evaluator.calculate_metrics(lstm_preds, lstm_truths)

            gru = GRUModel(input_size, **Config.model_config['GRU'], dropout=Config.dropout)
            gru_trainer = Trainer(gru, f'GRU_{setting["name"]}')
            gru_preds, gru_truths = gru_trainer.train(train_loader, test_loader)
            model_results['GRU'] = Evaluator.calculate_metrics(gru_preds, gru_truths)

            symbol_results[setting['name']] = model_results

            for model_name, preds in [('LSTM', lstm_preds), ('GRU', gru_preds)]:
                # 绘制预测结果
                plt.figure(figsize=(12, 6))
                plt.plot(lstm_truths, label='True Returns', alpha=0.7)
                plt.plot(preds, label=f'{model_name} {setting["name"]} Predicted Returns', linestyle='--')
                plt.title(f'{symbol} Return Prediction - {model_name} {setting["name"]}')
                plt.legend()
                plt.savefig(f'{symbol}_prediction_{setting["name"]}_{model_name}.png')
                plt.close()

                # 绘制ROC曲线
                pred_dir = (preds > 0).astype(int)
                true_dir = (lstm_truths > 0).astype(int)
                fpr, tpr, thresholds = roc_curve(true_dir, preds)
                roc_auc = auc(fpr, tpr)
                plt.figure(figsize=(8, 8))
                plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
                plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
                plt.xlim([0.0, 1.0])
                plt.ylim([0.0, 1.05])
                plt.xlabel('False Positive Rate')
                plt.ylabel('True Positive Rate')
                plt.title(f'{symbol} {model_name} {setting["name"]} ROC Curve')
                plt.legend(loc="lower right")
                plt.savefig(f'{symbol}_roc_curve_{setting["name"]}_{model_name}.png')
                plt.close()

        results[symbol] = symbol_results

    print("\n=== Final Results ===")
    for symbol in results:
        print(f"\n{symbol} Performance:")
        for setting in results[symbol]:
            print(f"\n  {setting} Setting:")
            for model in results[symbol][setting]:
                metrics = results[symbol][setting][model]
                print(f"    {model}:")
                print(f"      RMSE: {metrics['RMSE']:.4f} | MAE: {metrics['MAE']:.4f}")
                print(f"      Accuracy: {metrics['Accuracy']:.2%} | Recall: {metrics['Recall']:.2%}")
                print(f"      Sharpe: {metrics['Sharpe']:.2f} | Corr: {metrics['Correlation']:.2f}")
                print(f"      Precision: {metrics['Precision']:.2f}")
                print(f"      F1: {metrics['F1']:.2f}")
                print(f"      AUC: {metrics['AUC']:.2f}")
                print(f"      Confusion Matrix:\n{metrics['ConfusionMatrix']}")

if __name__ == '__main__':
    main()
    


=== Processing AAPL ===


[*********************100%***********************]  1 of 1 completed


Epoch 1/200 | Train Loss: 0.3989 | Test Loss: 0.5581
Epoch 11/200 | Train Loss: 0.0733 | Test Loss: 0.2325
Epoch 21/200 | Train Loss: 0.0477 | Test Loss: 0.1871
Epoch 31/200 | Train Loss: 0.0381 | Test Loss: 0.1899
Epoch 41/200 | Train Loss: 0.0342 | Test Loss: 0.1934
Early stopping at epoch 43
Epoch 1/200 | Train Loss: 0.3939 | Test Loss: 0.5424
Epoch 11/200 | Train Loss: 0.0637 | Test Loss: 0.2995
Epoch 21/200 | Train Loss: 0.0470 | Test Loss: 0.2647
Epoch 31/200 | Train Loss: 0.0396 | Test Loss: 0.2477
Epoch 41/200 | Train Loss: 0.0367 | Test Loss: 0.2370
Epoch 51/200 | Train Loss: 0.0329 | Test Loss: 0.2015
Epoch 61/200 | Train Loss: 0.0307 | Test Loss: 0.1970
Epoch 71/200 | Train Loss: 0.0277 | Test Loss: 0.1884
Early stopping at epoch 79


[*********************100%***********************]  1 of 1 completed


Epoch 1/200 | Train Loss: 0.4014 | Test Loss: 0.5575
Epoch 11/200 | Train Loss: 0.0763 | Test Loss: 0.2447
Epoch 21/200 | Train Loss: 0.0461 | Test Loss: 0.2044
Epoch 31/200 | Train Loss: 0.0386 | Test Loss: 0.2200
Early stopping at epoch 39
Epoch 1/200 | Train Loss: 0.3817 | Test Loss: 0.5398
Epoch 11/200 | Train Loss: 0.0664 | Test Loss: 0.2496
Epoch 21/200 | Train Loss: 0.0462 | Test Loss: 0.2298
Epoch 31/200 | Train Loss: 0.0392 | Test Loss: 0.2275
Epoch 41/200 | Train Loss: 0.0342 | Test Loss: 0.1948
Epoch 51/200 | Train Loss: 0.0321 | Test Loss: 0.1859
Epoch 61/200 | Train Loss: 0.0288 | Test Loss: 0.1753
Epoch 71/200 | Train Loss: 0.0255 | Test Loss: 0.1578
Epoch 81/200 | Train Loss: 0.0232 | Test Loss: 0.1518
Epoch 91/200 | Train Loss: 0.0222 | Test Loss: 0.1481
Epoch 101/200 | Train Loss: 0.0203 | Test Loss: 0.1447
Epoch 111/200 | Train Loss: 0.0201 | Test Loss: 0.1379
Epoch 121/200 | Train Loss: 0.0188 | Test Loss: 0.1397
Epoch 131/200 | Train Loss: 0.0170 | Test Loss: 0.1388

...

Epoch 1/200 | Train Loss: 0.3646 | Test Loss: 0.5111
Epoch 11/200 | Train Loss: 0.0689 | Test Loss: 0.1867
Epoch 21/200 | Train Loss: 0.0412 | Test Loss: 0.1401
Epoch 31/200 | Train Loss: 0.0336 | Test Loss: 0.1256
Epoch 41/200 | Train Loss: 0.0284 | Test Loss: 0.1244
Epoch 51/200 | Train Loss: 0.0255 | Test Loss: 0.1241
Early stopping at epoch 60
Epoch 1/200 | Train Loss: 0.3702 | Test Loss: 0.5312
Epoch 11/200 | Train Loss: 0.0643 | Test Loss: 0.1519
Epoch 21/200 | Train Loss: 0.0407 | Test Loss: 0.1121
Epoch 31/200 | Train Loss: 0.0334 | Test Loss: 0.0983
Epoch 41/200 | Train Loss: 0.0301 | Test Loss: 0.0878
Epoch 51/200 | Train Loss: 0.0268 | Test Loss: 0.0802
Epoch 61/200 | Train Loss: 0.0249 | Test Loss: 0.0783
Epoch 71/200 | Train Loss: 0.0228 | Test Loss: 0.0762
Epoch 81/200 | Train Loss: 0.0222 | Test Loss: 0.0735
Epoch 91/200 | Train Loss: 0.0207 | Test Loss: 0.0750
Early stopping at epoch 95


[*********************100%***********************]  1 of 1 completed


Epoch 1/200 | Train Loss: 0.3655 | Test Loss: 0.5135
Epoch 11/200 | Train Loss: 0.1185 | Test Loss: 0.1892
Epoch 21/200 | Train Loss: 0.1030 | Test Loss: 0.1634
Epoch 31/200 | Train Loss: 0.0949 | Test Loss: 0.1611
Epoch 41/200 | Train Loss: 0.0929 | Test Loss: 0.1602
Epoch 51/200 | Train Loss: 0.0868 | Test Loss: 0.1609
Early stopping at epoch 58
Epoch 1/200 | Train Loss: 0.3711 | Test Loss: 0.5160
Epoch 11/200 | Train Loss: 0.1138 | Test Loss: 0.1672
Epoch 21/200 | Train Loss: 0.1018 | Test Loss: 0.1577
Epoch 31/200 | Train Loss: 0.0991 | Test Loss: 0.1583
Epoch 41/200 | Train Loss: 0.0925 | Test Loss: 0.1509
Epoch 51/200 | Train Loss: 0.0933 | Test Loss: 0.1488
Epoch 61/200 | Train Loss: 0.0899 | Test Loss: 0.1459
Epoch 71/200 | Train Loss: 0.0884 | Test Loss: 0.1453
Epoch 81/200 | Train Loss: 0.0847 | Test Loss: 0.1428
Epoch 91/200 | Train Loss: 0.0827 | Test Loss: 0.1437
Epoch 101/200 | Train Loss: 0.0811 | Test Loss: 0.1449
Early stopping at epoch 103


[*********************100%***********************]  1 of 1 completed


Epoch 1/200 | Train Loss: 0.3666 | Test Loss: 0.5178
Epoch 11/200 | Train Loss: 0.0761 | Test Loss: 0.1666
Epoch 21/200 | Train Loss: 0.0413 | Test Loss: 0.1087
Epoch 31/200 | Train Loss: 0.0312 | Test Loss: 0.0959
Epoch 41/200 | Train Loss: 0.0304 | Test Loss: 0.0915
Epoch 51/200 | Train Loss: 0.0268 | Test Loss: 0.0895
Epoch 61/200 | Train Loss: 0.0257 | Test Loss: 0.0872
Epoch 71/200 | Train Loss: 0.0231 | Test Loss: 0.0933
Early stopping at epoch 75
Epoch 1/200 | Train Loss: 0.3512 | Test Loss: 0.5009
Epoch 11/200 | Train Loss: 0.0611 | Test Loss: 0.1476
Epoch 21/200 | Train Loss: 0.0408 | Test Loss: 0.1047
Epoch 31/200 | Train Loss: 0.0348 | Test Loss: 0.0896
Epoch 41/200 | Train Loss: 0.0299 | Test Loss: 0.0825
Epoch 51/200 | Train Loss: 0.0278 | Test Loss: 0.0819
Epoch 61/200 | Train Loss: 0.0262 | Test Loss: 0.0763
Epoch 71/200 | Train Loss: 0.0243 | Test Loss: 0.0795
Epoch 81/200 | Train Loss: 0.0228 | Test Loss: 0.0789
Early stopping at epoch 84

=== Final Results ===

AAPL P

In [5]:
import csv

# Data for each stock
stocks_data = {
    "AAPL": [
        {"Model": "LSTM", "Setting": "Full", "RMSE": 0.4581, "MAE": 0.2982, "Accuracy (%)": 87.92, "Recall (%)": 84.88, "Sharpe": -2.37, "Corr": 0.90, "Precision": 0.91, "F1": 0.88, "AUC": 0.96, "Confusion Matrix": "[[290, 28], [52, 292]]"},
        {"Model": "GRU", "Setting": "Full", "RMSE": 0.4781, "MAE": 0.3423, "Accuracy (%)": 87.16, "Recall (%)": 82.56, "Sharpe": -2.53, "Corr": 0.90, "Precision": 0.92, "F1": 0.87, "AUC": 0.95, "Confusion Matrix": "[[293, 25], [60, 284]]"},
        {"Model": "LSTM", "Setting": "No_PE", "RMSE": 0.4919, "MAE": 0.3309, "Accuracy (%)": 86.25, "Recall (%)": 79.36, "Sharpe": -3.83, "Corr": 0.89, "Precision": 0.93, "F1": 0.86, "AUC": 0.95, "Confusion Matrix": "[[298, 20], [71, 273]]"},
        {"Model": "GRU", "Setting": "No_PE", "RMSE": 0.3914, "MAE": 0.2745, "Accuracy (%)": 90.33, "Recall (%)": 87.21, "Sharpe": -2.06, "Corr": 0.93, "Precision": 0.94, "F1": 0.90, "AUC": 0.97, "Confusion Matrix": "[[298, 20], [44, 300]]"},
        {"Model": "LSTM", "Setting": "No_IMF", "RMSE": 0.5218, "MAE": 0.3799, "Accuracy (%)": 84.29, "Recall (%)": 81.98, "Sharpe": -1.68, "Corr": 0.86, "Precision": 0.87, "F1": 0.84, "AUC": 0.92, "Confusion Matrix": "[[276, 42], [62, 282]]"},
        {"Model": "GRU", "Setting": "No_IMF", "RMSE": 0.4887, "MAE": 0.3558, "Accuracy (%)": 84.14, "Recall (%)": 81.98, "Sharpe": -1.23, "Corr": 0.88, "Precision": 0.87, "F1": 0.84, "AUC": 0.92, "Confusion Matrix": "[[275, 43], [62, 282]]"},
        {"Model": "LSTM", "Setting": "No_Tech", "RMSE": 0.3993, "MAE": 0.2341, "Accuracy (%)": 91.09, "Recall (%)": 89.83, "Sharpe": -1.89, "Corr": 0.92, "Precision": 0.93, "F1": 0.91, "AUC": 0.97, "Confusion Matrix": "[[294, 24], [35, 309]]"},
        {"Model": "GRU", "Setting": "No_Tech", "RMSE": 0.3529, "MAE": 0.2104, "Accuracy (%)": 91.69, "Recall (%)": 90.41, "Sharpe": -1.53, "Corr": 0.94, "Precision": 0.93, "F1": 0.92, "AUC": 0.97, "Confusion Matrix": "[[296, 22], [33, 311]]"},
    ],
    "MSFT": [
        {"Model": "LSTM", "Setting": "Full", "RMSE": 0.5393, "MAE": 0.3387, "Accuracy (%)": 87.46, "Recall (%)": 88.27, "Sharpe": 0.40, "Corr": 0.85, "Precision": 0.88, "F1": 0.88, "AUC": 0.94, "Confusion Matrix": "[[278, 43], [40, 301]]"},
        {"Model": "GRU", "Setting": "Full", "RMSE": 0.4914, "MAE": 0.2905, "Accuracy (%)": 90.18, "Recall (%)": 87.39, "Sharpe": -0.80, "Corr": 0.88, "Precision": 0.93, "F1": 0.90, "AUC": 0.96, "Confusion Matrix": "[[299, 22], [43, 298]]"},
        {"Model": "LSTM", "Setting": "No_PE", "RMSE": 0.5224, "MAE": 0.3213, "Accuracy (%)": 88.52, "Recall (%)": 86.22, "Sharpe": -0.48, "Corr": 0.86, "Precision": 0.91, "F1": 0.89, "AUC": 0.95, "Confusion Matrix": "[[292, 29], [47, 294]]"},
        {"Model": "GRU", "Setting": "No_PE", "RMSE": 0.4836, "MAE": 0.2850, "Accuracy (%)": 90.18, "Recall (%)": 87.68, "Sharpe": -0.65, "Corr": 0.88, "Precision": 0.93, "F1": 0.90, "AUC": 0.96, "Confusion Matrix": "[[298, 23], [42, 299]]"},
        {"Model": "LSTM", "Setting": "No_IMF", "RMSE": 0.5986, "MAE": 0.4704, "Accuracy (%)": 82.33, "Recall (%)": 72.73, "Sharpe": -3.62, "Corr": 0.84, "Precision": 0.91, "F1": 0.81, "AUC": 0.92, "Confusion Matrix": "[[297, 24], [93, 248]]"},
        {"Model": "GRU", "Setting": "No_IMF", "RMSE": 0.5409, "MAE": 0.4148, "Accuracy (%)": 83.08, "Recall (%)": 82.40, "Sharpe": -0.67, "Corr": 0.85, "Precision": 0.84, "F1": 0.83, "AUC": 0.91, "Confusion Matrix": "[[269, 52], [60, 281]]"},
        {"Model": "LSTM", "Setting": "No_Tech", "RMSE": 0.4782, "MAE": 0.2932, "Accuracy (%)": 90.18, "Recall (%)": 88.86, "Sharpe": -0.47, "Corr": 0.89, "Precision": 0.92, "F1": 0.90, "AUC": 0.96, "Confusion Matrix": "[[294, 27], [38, 303]]"},
        {"Model": "GRU", "Setting": "No_Tech", "RMSE": 0.4681, "MAE": 0.2740, "Accuracy (%)": 90.79, "Recall (%)": 89.44, "Sharpe": -0.66, "Corr": 0.89, "Precision": 0.92, "F1": 0.91, "AUC": 0.96, "Confusion Matrix": "[[296, 25], [36, 305]]"},
    ],
    "AMZN": [
        {"Model": "LSTM", "Setting": "Full", "RMSE": 0.4405, "MAE": 0.2972, "Accuracy (%)": 86.71, "Recall (%)": 84.07, "Sharpe": -1.83, "Corr": 0.92, "Precision": 0.89, "F1": 0.87, "AUC": 0.95, "Confusion Matrix": "[[289, 34], [54, 285]]"},
        {"Model": "GRU", "Setting": "Full", "RMSE": 0.3918, "MAE": 0.2663, "Accuracy (%)": 89.58, "Recall (%)": 90.56, "Sharpe": -0.77, "Corr": 0.93, "Precision": 0.89, "F1": 0.90, "AUC": 0.96, "Confusion Matrix": "[[286, 37], [32, 307]]"},
        {"Model": "LSTM", "Setting": "No_PE", "RMSE": 0.4347, "MAE": 0.2924, "Accuracy (%)": 86.86, "Recall (%)": 85.55, "Sharpe": -1.60, "Corr": 0.92, "Precision": 0.88, "F1": 0.87, "AUC": 0.95, "Confusion Matrix": "[[285, 38], [49, 290]]"},
        {"Model": "GRU", "Setting": "No_PE", "RMSE": 0.3983, "MAE": 0.2680, "Accuracy (%)": 88.52, "Recall (%)": 87.91, "Sharpe": -0.77, "Corr": 0.93, "Precision": 0.89, "F1": 0.89, "AUC": 0.96, "Confusion Matrix": "[[288, 35], [41, 298]]"},
        {"Model": "LSTM", "Setting": "No_IMF", "RMSE": 0.5409, "MAE": 0.4152, "Accuracy (%)": 82.33, "Recall (%)": 79.35, "Sharpe": -1.22, "Corr": 0.87, "Precision": 0.85, "F1": 0.82, "AUC": 0.92, "Confusion Matrix": "[[276, 47], [70, 269]]"},
        {"Model": "GRU", "Setting": "No_IMF", "RMSE": 0.5267, "MAE": 0.4031, "Accuracy (%)": 81.12, "Recall (%)": 79.35, "Sharpe": -0.79, "Corr": 0.87, "Precision": 0.83, "F1": 0.81, "AUC": 0.92, "Confusion Matrix": "[[268, 55], [70, 269]]"},
        {"Model": "LSTM", "Setting": "No_Tech", "RMSE": 0.4247, "MAE": 0.2807, "Accuracy (%)": 88.52, "Recall (%)": 86.14, "Sharpe": -1.62, "Corr": 0.92, "Precision": 0.91, "F1": 0.88, "AUC": 0.96, "Confusion Matrix": "[[294, 29], [47, 292]]"},
        {"Model": "GRU", "Setting": "No_Tech", "RMSE": 0.3906, "MAE": 0.2639, "Accuracy (%)": 89.27, "Recall (%)": 89.09, "Sharpe": -0.78, "Corr": 0.93, "Precision": 0.90, "F1": 0.89, "AUC": 0.96, "Confusion Matrix": "[[289, 34], [37, 302]]"},
    ],
    "GOOG": [
        {"Model": "LSTM", "Setting": "Full", "RMSE": 0.4785, "MAE": 0.3349, "Accuracy (%)": 89.27, "Recall (%)": 89.22, "Sharpe": -0.28, "Corr": 0.92, "Precision": 0.89, "F1": 0.89, "AUC": 0.95, "Confusion Matrix": "[[293, 35], [36, 298]]"},
        {"Model": "GRU", "Setting": "Full", "RMSE": 0.4237, "MAE": 0.2945, "Accuracy (%)": 89.27, "Recall (%)": 88.02, "Sharpe": -0.67, "Corr": 0.93, "Precision": 0.90, "F1": 0.89, "AUC": 0.97, "Confusion Matrix": "[[297, 31], [40, 294]]"},
        {"Model": "LSTM", "Setting": "No_PE", "RMSE": 0.4659, "MAE": 0.3285, "Accuracy (%)": 89.88, "Recall (%)": 87.43, "Sharpe": -0.88, "Corr": 0.92, "Precision": 0.92, "F1": 0.90, "AUC": 0.96, "Confusion Matrix": "[[303, 25], [42, 292]]"},
        {"Model": "GRU", "Setting": "No_PE", "RMSE": 0.4246, "MAE": 0.2986, "Accuracy (%)": 89.73, "Recall (%)": 85.93, "Sharpe": -1.50, "Corr": 0.94, "Precision": 0.93, "F1": 0.89, "AUC": 0.97, "Confusion Matrix": "[[307, 21], [47, 287]]"},
        {"Model": "LSTM", "Setting": "No_IMF", "RMSE": 0.5939, "MAE": 0.4516, "Accuracy (%)": 83.23, "Recall (%)": 82.34, "Sharpe": -0.51, "Corr": 0.87, "Precision": 0.83, "F1": 0.83, "AUC": 0.92, "Confusion Matrix": "[[276, 52], [59, 275]]"},
        {"Model": "GRU", "Setting": "No_IMF", "RMSE": 0.5973, "MAE": 0.4549, "Accuracy (%)": 83.53, "Recall (%)": 82.93, "Sharpe": -0.0, "Corr": 0.86, "Precision": 0.84, "F1": 0.84, "AUC": 0.92, "Confusion Matrix": "[[276, 52], [57, 277]]"},
        {"Model": "LSTM", "Setting": "No_Tech", "RMSE": 0.4510, "MAE": 0.3129, "Accuracy (%)": 90.33, "Recall (%)": 88.02, "Sharpe": -1.19, "Corr": 0.93, "Precision": 0.92, "F1": 0.90, "AUC": 0.96, "Confusion Matrix": "[[304, 24], [40, 294]]"},
        {"Model": "GRU", "Setting": "No_Tech", "RMSE": 0.4135, "MAE": 0.2874, "Accuracy (%)": 91.54, "Recall (%)": 89.52, "Sharpe": -1.04, "Corr": 0.94, "Precision": 0.93, "F1": 0.91, "AUC": 0.97, "Confusion Matrix": "[[307, 21], [35, 299]]"},
    ],
    "META": [
        {"Model": "LSTM", "Setting": "Full", "RMSE": 0.4346, "MAE": 0.3053, "Accuracy (%)": 87.25, "Recall (%)": 80.21, "Sharpe": -0.60, "Corr": 0.93, "Precision": 0.97, "F1": 0.88, "AUC": 0.95, "Confusion Matrix": "[[261, 8], [74, 300]]"},
        {"Model": "GRU", "Setting": "Full", "RMSE": 0.3501, "MAE": 0.2455, "Accuracy (%)": 89.74, "Recall (%)": 88.50, "Sharpe": 1.35, "Corr": 0.95, "Precision": 0.94, "F1": 0.91, "AUC": 0.97, "Confusion Matrix": "[[246, 23], [43, 331]]"},
        {"Model": "LSTM", "Setting": "No_PE", "RMSE": 0.3930, "MAE": 0.2655, "Accuracy (%)": 90.05, "Recall (%)": 89.84, "Sharpe": 1.59, "Corr": 0.94, "Precision": 0.93, "F1": 0.91, "AUC": 0.96, "Confusion Matrix": "[[243, 26], [38, 336]]"},
        {"Model": "GRU", "Setting": "No_PE", "RMSE": 0.3575, "MAE": 0.2525, "Accuracy (%)": 89.42, "Recall (%)": 87.17, "Sharpe": 1.25, "Corr": 0.95, "Precision": 0.94, "F1": 0.91, "AUC": 0.97, "Confusion Matrix": "[[249, 20], [48, 326]]"},
        {"Model": "LSTM", "Setting": "No_IMF", "RMSE": 0.5370, "MAE": 0.3949, "Accuracy (%)": 82.89, "Recall (%)": 80.75, "Sharpe": -0.25, "Corr": 0.89, "Precision": 0.89, "F1": 0.85, "AUC": 0.90, "Confusion Matrix": "[[231, 38], [72, 302]]"},
        {"Model": "GRU", "Setting": "No_IMF", "RMSE": 0.5399, "MAE": 0.3913, "Accuracy (%)": 83.83, "Recall (%)": 82.62, "Sharpe": 0.67, "Corr": 0.88, "Precision": 0.89, "F1": 0.86, "AUC": 0.91, "Confusion Matrix": "[[230, 39], [65, 309]]"},
        {"Model": "LSTM", "Setting": "No_Tech", "RMSE": 0.3460, "MAE": 0.2435, "Accuracy (%)": 90.51, "Recall (%)": 89.30, "Sharpe": 1.42, "Corr": 0.95, "Precision": 0.94, "F1": 0.92, "AUC": 0.97, "Confusion Matrix": "[[248, 21], [40, 334]]"},
        {"Model": "GRU", "Setting": "No_Tech", "RMSE": 0.3456, "MAE": 0.2421, "Accuracy (%)": 89.74, "Recall (%)": 91.18, "Sharpe": 1.88, "Corr": 0.95, "Precision": 0.91, "F1": 0.91, "AUC": 0.97, "Confusion Matrix": "[[236, 33], [33, 341]]"},
    ],
    "NVDA": [
        {"Model": "LSTM", "Setting": "Full", "RMSE": 0.5075, "MAE": 0.3644, "Accuracy (%)": 85.80, "Recall (%)": 87.67, "Sharpe": 1.75, "Corr": 0.90, "Precision": 0.87, "F1": 0.87, "AUC": 0.95, "Confusion Matrix": "[[248, 49], [45, 320]]"},
        {"Model": "GRU", "Setting": "Full", "RMSE": 0.4435, "MAE": 0.3200, "Accuracy (%)": 88.07, "Recall (%)": 88.77, "Sharpe": 1.53, "Corr": 0.93, "Precision": 0.90, "F1": 0.89, "AUC": 0.96, "Confusion Matrix": "[[259, 38], [41, 324]]"},
        {"Model": "LSTM", "Setting": "No_PE", "RMSE": 0.5291, "MAE": 0.3848, "Accuracy (%)": 85.95, "Recall (%)": 88.77, "Sharpe": 2.76, "Corr": 0.89, "Precision": 0.86, "F1": 0.87, "AUC": 0.95, "Confusion Matrix": "[[245, 52], [41, 324]]"},
        {"Model": "GRU", "Setting": "No_PE", "RMSE": 0.4524, "MAE": 0.3148, "Accuracy (%)": 88.52, "Recall (%)": 90.41, "Sharpe": 2.04, "Corr": 0.92, "Precision": 0.89, "F1": 0.90, "AUC": 0.96, "Confusion Matrix": "[[256, 41], [35, 330]]"},
        {"Model": "LSTM", "Setting": "No_IMF", "RMSE": 0.6738, "MAE": 0.5225, "Accuracy (%)": 82.48, "Recall (%)": 78.36, "Sharpe": -1.99, "Corr": 0.85, "Precision": 0.89, "F1": 0.83, "AUC": 0.91, "Confusion Matrix": "[[260, 37], [79, 286]]"},
        {"Model": "GRU", "Setting": "No_IMF", "RMSE": 0.6740, "MAE": 0.5142, "Accuracy (%)": 84.44, "Recall (%)": 80.55, "Sharpe": -1.56, "Corr": 0.84, "Precision": 0.90, "F1": 0.85, "AUC": 0.91, "Confusion Matrix": "[[265, 32], [71, 294]]"},
        {"Model": "LSTM", "Setting": "No_Tech", "RMSE": 0.4418, "MAE": 0.3043, "Accuracy (%)": 88.82, "Recall (%)": 88.22, "Sharpe": 0.76, "Corr": 0.93, "Precision": 0.91, "F1": 0.90, "AUC": 0.96, "Confusion Matrix": "[[266, 31], [43, 322]]"},
        {"Model": "GRU", "Setting": "No_Tech", "RMSE": 0.4064, "MAE": 0.2818, "Accuracy (%)": 89.43, "Recall (%)": 90.68, "Sharpe": 1.42, "Corr": 0.94, "Precision": 0.90, "F1": 0.90, "AUC": 0.96, "Confusion Matrix": "[[261, 36], [34, 331]]"},
    ],
    "TSLA": [
        {"Model": "LSTM", "Setting": "Full", "RMSE": 0.4948, "MAE": 0.3921, "Accuracy (%)": 87.01, "Recall (%)": 84.67, "Sharpe": -3.25, "Corr": 0.90, "Precision": 0.86, "F1": 0.86, "AUC": 0.94, "Confusion Matrix": "[[322, 40], [46, 254]]"},
        {"Model": "GRU", "Setting": "Full", "RMSE": 0.3154, "MAE": 0.2397, "Accuracy (%)": 88.82, "Recall (%)": 91.67, "Sharpe": -0.32, "Corr": 0.96, "Precision": 0.85, "F1": 0.88, "AUC": 0.97, "Confusion Matrix": "[[313, 49], [25, 275]]"},
        {"Model": "LSTM", "Setting": "No_PE", "RMSE": 0.4456, "MAE": 0.3373, "Accuracy (%)": 87.46, "Recall (%)": 86.00, "Sharpe": -2.38, "Corr": 0.92, "Precision": 0.86, "F1": 0.86, "AUC": 0.95, "Confusion Matrix": "[[321, 41], [42, 258]]"},
        {"Model": "GRU", "Setting": "No_PE", "RMSE": 0.3229, "MAE": 0.2503, "Accuracy (%)": 87.92, "Recall (%)": 92.00, "Sharpe": -0.06, "Corr": 0.96, "Precision": 0.83, "F1": 0.87, "AUC": 0.97, "Confusion Matrix": "[[306, 56], [24, 276]]"},
        {"Model": "LSTM", "Setting": "No_IMF", "RMSE": 0.5408, "MAE": 0.4157, "Accuracy (%)": 82.78, "Recall (%)": 80.67, "Sharpe": -1.17, "Corr": 0.87, "Precision": 0.81, "F1": 0.81, "AUC": 0.91, "Confusion Matrix": "[[306, 56], [58, 242]]"},
        {"Model": "GRU", "Setting": "No_IMF", "RMSE": 0.5164, "MAE": 0.3957, "Accuracy (%)": 83.69, "Recall (%)": 81.00, "Sharpe": -1.17, "Corr": 0.88, "Precision": 0.83, "F1": 0.82, "AUC": 0.92, "Confusion Matrix": "[[311, 51], [57, 243]]"},
        {"Model": "LSTM", "Setting": "No_Tech", "RMSE": 0.3787, "MAE": 0.2870, "Accuracy (%)": 88.67, "Recall (%)": 92.00, "Sharpe": -0.55, "Corr": 0.95, "Precision": 0.84, "F1": 0.88, "AUC": 0.97, "Confusion Matrix": "[[311, 51], [24, 276]]"},
        {"Model": "GRU", "Setting": "No_Tech", "RMSE": 0.3378, "MAE": 0.2537, "Accuracy (%)": 89.88, "Recall (%)": 91.67, "Sharpe": -0.28, "Corr": 0.96, "Precision": 0.87, "F1": 0.89, "AUC": 0.97, "Confusion Matrix": "[[320, 42], [25, 275]]"},
    ]
}

# CSV headers
headers = ["Model", "Setting", "RMSE", "MAE", "Accuracy (%)", "Recall (%)", "Sharpe", "Corr", "Precision", "F1", "AUC", "Confusion Matrix"]

# Generate a CSV file for each stock
for stock, data in stocks_data.items():
    filename = f"{stock}_lstm_gru_performance.csv"
    try:
        with open(filename, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.DictWriter(file, fieldnames=headers)
            writer.writeheader()
            for row in data:
                writer.writerow(row)
        print(f"CSV file generated for {stock}: {filename}")
    except Exception as e:
        print(f"Error generating CSV file for {stock}: {e}")
    

CSV file generated for AAPL: AAPL_lstm_gru_performance.csv
CSV file generated for MSFT: MSFT_lstm_gru_performance.csv
CSV file generated for AMZN: AMZN_lstm_gru_performance.csv
CSV file generated for GOOG: GOOG_lstm_gru_performance.csv
CSV file generated for META: META_lstm_gru_performance.csv
CSV file generated for NVDA: NVDA_lstm_gru_performance.csv
CSV file generated for TSLA: TSLA_lstm_gru_performance.csv
